In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [6]:
# 1. 读取数据
data = pd.read_csv('D:\课题会\丹参\danshen_code\SDAE\SDAE-main\data\丹参数据salvia_all_20240425 - 副本.csv')  # 替换为你的数据文件路径

# 2. 删除前三列
data = data.iloc[:, 3:]
# 3. 删除指定目标变量
drop_cols = [
    'CS','MT','TSIIA','TSI','DTSI','SumTS','PD','CFA','FA','SAD','SF','DSS',
    'SAC','SAE','MCF','RA','SAA','LA','SAY','TA','CTA','MA','FMA','SUA','SAB',"Soil_sampleN",'etestN','testNp','etestpatch'
]
data = data.drop(columns=[col for col in drop_cols if col in data.columns])

# 4. 删除空白值较多的列
thresh = len(data) * 0.5
data = data.dropna(axis=1, thresh=thresh)

# 5. 用均值/众数填充剩余缺失值
for col in data.select_dtypes(include=['float64', 'int64']).columns:
    data[col] = data[col].fillna(data[col].mean())
for col in data.select_dtypes(include=['object']).columns:
    data[col] = data[col].fillna(data[col].mode()[0])
    
# 6. 指定需要独热编码的分类特征列
categorical_columns = [
    "Province",
    "City",
    "Microb",
    "etestN",
    "etestpatch",
    "testNp",
    "Landscape",
    "SoilType",
    "soilclass",      
    "CultivationType",
    "ClimateType",
    "按气候聚类划分的类型"
]
# 只对存在于数据中的列做独热编码
categorical_columns = [col for col in categorical_columns if col in data.columns]
data = pd.get_dummies(data, columns=categorical_columns, drop_first=True)

# 7. 标准化
features = data.select_dtypes(include=['float64', 'int64'])
scaler = StandardScaler()
X_scaled = scaler.fit_transform(features)

# 3. 聚类分析
k = 3  # 你可以根据实际情况调整聚类数
kmeans = KMeans(n_clusters=k, random_state=42)
labels = kmeans.fit_predict(X_scaled)

# 4. 可视化（PCA降维到2维）
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)
plt.figure(figsize=(8,6))
for i in range(k):
    plt.scatter(X_pca[labels==i, 0], X_pca[labels==i, 1], label=f'Cluster {i+1}')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('KMeans聚类结果（PCA降维可视化）')
plt.legend()
plt.show()

# 5. 聚类标签保存
data['Cluster'] = labels
data.to_csv('clustered_data.csv', index=False)

ValueError: Input X contains NaN.
KMeans does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values